In [46]:
import os
import json
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path
import numpy as np

In [47]:
root_path = Path('./data/random/')
results = {}

for subdir, dirs, files in os.walk(root_path):
    for file in files:
        if file == 'results.json':
            print(os.path.join(subdir, file))
            _, query, params, seed = subdir.split('/')
            if query not in results:
                results[query] = {}

            if params not in results[query]:
                results[query][params] = {}

            with open(subdir + '/' + 'results.json', 'r') as f:
                res = json.load(f)
            results[query][params][seed] = res

data/random/lr0.01_wd0.005/seed1/results.json
data/random/lr0.01_wd0.005/seed2/results.json
data/random/lr0.001_wd0.005/seed1/results.json
data/random/lr0.001_wd0.005/seed2/results.json
data/random/lr0.001_wd0.05/seed1/results.json
data/random/lr0.001_wd0.05/seed2/results.json
data/random/lr0.001_wd0.0005/seed1/results.json
data/random/lr0.001_wd0.0005/seed2/results.json
data/random/lr0.01_wd0.0005/seed1/results.json
data/random/lr0.01_wd0.0005/seed2/results.json
data/random/lr0.01_wd0.05/seed1/results.json
data/random/lr0.01_wd0.05/seed2/results.json
data/random/lr0.1_wd0.005/seed1/results.json
data/random/lr0.1_wd0.005/seed2/results.json
data/random/lr0.1_wd0.05/seed1/results.json
data/random/lr0.1_wd0.05/seed2/results.json
data/random/lr0.1_wd0.0005/seed1/results.json
data/random/lr0.1_wd0.0005/seed2/results.json


In [48]:
test_accuracys = {}

for query in results:
    test_accuracys[query] = {}
    for params in results[query]:
        test_accuracys[query][params] = []
        for seed in results[query][params]:
            # Find optimal run
            best_acc = -1
            for trial in results[query][params][seed]['ray-results']:
                if 'test_acc' in trial['res']:
                    if trial['res']['test_acc'] > best_acc:
                        best_acc = trial['res']['test_acc']
            test_accuracys[query][params].append(best_acc*100)

In [49]:
test_accuracys

{'random': {'lr0.01_wd0.005': [83.49999785423279, 83.48000049591064],
  'lr0.001_wd0.005': [83.52000117301941, 83.27000141143799],
  'lr0.001_wd0.05': [82.96999931335449, 83.31000208854675],
  'lr0.001_wd0.0005': [82.95000195503235, 83.05000066757202],
  'lr0.01_wd0.0005': [83.06999802589417, 83.12000036239624],
  'lr0.01_wd0.05': [83.27000141143799, 83.17999839782715],
  'lr0.1_wd0.005': [83.37000012397766, 83.17999839782715],
  'lr0.1_wd0.05': [83.24000239372253, 83.45000147819519],
  'lr0.1_wd0.0005': [83.56000185012817, 83.31000208854675]}}

In [50]:
df = {}
for query in test_accuracys:
    df[query] = {}
    for params in test_accuracys[query]:
        df[query][params] = str(np.mean(test_accuracys[query][params]).round(4)) + ' +/- ' + str(np.std(test_accuracys[query][params]).round(4))
pd.DataFrame(df)

,random
lr0.001_wd0.0005,83.0 +/- 0.05
lr0.001_wd0.005,83.395 +/- 0.125
lr0.001_wd0.05,83.14 +/- 0.17
lr0.01_wd0.0005,83.095 +/- 0.025
lr0.01_wd0.005,83.49 +/- 0.01
lr0.01_wd0.05,83.225 +/- 0.045
lr0.1_wd0.0005,83.435 +/- 0.125
lr0.1_wd0.005,83.275 +/- 0.095
lr0.1_wd0.05,83.345 +/- 0.105
